In [1]:
# install needed packages
import pygsheets
import pandas as pd
import requests
import itertools
from bs4 import BeautifulSoup
from random import randint
from pytrends.request import TrendReq

In [2]:
# authorization
# use creds to create a client to interact with the Google Drive API
gc = pygsheets.authorize(outh_file='C:/Users/Thatshelbs/client_secret.json')

In [3]:
#Start with google trends
pytrend = TrendReq()

In [4]:
kw_list=["Kirk Cousins","Sam Bradford","Teddy Bridgewater","Case Keenum"]
pytrend.build_payload(kw_list,geo='US')

In [5]:
interest_over_time_df = pytrend.interest_over_time()
#end google trends

In [6]:
#some prep for google sheets
#it didn't like the date format
#reshaping dataframe for tableau
ts = pd.to_datetime(interest_over_time_df.index)

In [7]:
interest_over_time_df['date'] = ts.strftime('%Y.%m.%d')
interest_over_time_df = interest_over_time_df.loc[interest_over_time_df['isPartial'] == 'False']
interest_over_time_df = interest_over_time_df[['date',"Kirk Cousins","Sam Bradford","Teddy Bridgewater","Case Keenum"]]

In [8]:
interest_over_time_df2 = pd.melt(interest_over_time_df,id_vars=['date'],var_name='qb_name',value_name='index')

In [9]:
# Open Google spreadsheet and then workseet
sh = gc.open('Butterfly Kneeffect')
wks = sh.worksheet_by_title("gt_data")

#set the values of a pandas dataframe to sheet
wks.set_dataframe(interest_over_time_df2,(1,1))

#Google trends has now been uploaded to google sheets account

In [10]:
#I wanted to scrape espn.com quarterback rating (QBR) results from tables on their website
year = list(range(2013,2019))
week = list(range(1,17))
week_year = list(itertools.product(year,week))

In [11]:
qb_list = []
sep = ','
y=0
for x in week_year:
    r = requests.get("http://www.espn.com/nfl/qbr/_/year/%s/type/player-week/week/%s" % (str(week_year[y][0]),str(week_year[y][1])))
    soup = BeautifulSoup(r.content, 'html.parser')
    for tr in soup.find_all('tr'):
        tds = tr.find_all('td')
        qb_name = tds[1].text.split(sep)[0]
        qbr = tds[10].text.strip()
        year2 = week_year[y][0]
        week2 = week_year[y][1]
        qb_row = {'year' : year2,'week' : week2, 'qb_name' : qb_name,'qbr':qbr}
        if qb_row['qb_name'] in ['Sam Bradford', 'Case Keenum', 'Teddy Bridgewater', 'Kirk Cousins']:
            qb_list.append(qb_row)
    y=y+1
    


In [12]:
#pandas
qb_list_df = pd.DataFrame(qb_list)

# Open spreadsheet and then worksheet
wks2 = sh.worksheet_by_title("qbr_data")

#set the values of a pandas dataframe to sheet
wks2.set_dataframe(qb_list_df,(1,1))

#webscrapped QBR now in google sheets account
